In [10]:
%pip install fastparquet

import pandas as pd

You should consider upgrading via the '/home/ramin/Developer/venv/salsa-evaluation/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [69]:
# Parameters
data = "../../salsa/data/single_partition/partition_0/left_index.gzip"
sample = 4
min_interactions = 10
output = "../data/users.json"

In [70]:
# Import users from dataset
keep_columns = [0, 1]

df = pd.read_parquet(data, columns=['adjacency_list', 'user_id'])
df.reset_index(level=0, inplace=True)

In [71]:
df["len_adjacency_list"] = df["adjacency_list"].str.len()
active_users = df[df["len_adjacency_list"] >= min_interactions]

In [72]:
# Sample users
if sample:
    active_users = active_users.sample(n=sample)

active_users

,user_id,adjacency_list,len_adjacency_list
2553955,1171995686260183043,"[1367303245622767619, 1366739413942738946, 136...",12
1249004,2711518648,"[1358829219313238018, 1367314260846469125, 136...",26
3528829,1366766574003183622,"[1367203640927268864, 1367436501135609857, 136...",26
2127683,1004470385859891200,"[1367475641680080898, 1367220355908202496, 136...",24


In [73]:
# Save results
active_users["user_id"].to_json(output, orient="table", index=False)
active_users.reset_index(drop=True)

,user_id,adjacency_list,len_adjacency_list
0,1171995686260183043,"[1367303245622767619, 1366739413942738946, 136...",12
1,2711518648,"[1358829219313238018, 1367314260846469125, 136...",26
2,1366766574003183622,"[1367203640927268864, 1367436501135609857, 136...",26
3,1004470385859891200,"[1367475641680080898, 1367220355908202496, 136...",24


In [80]:
# active_users = active_users.set_index('user_id')
df = active_users["adjacency_list"].apply(pd.Series).unstack().reset_index(level=0, drop=True).dropna()
df = df.to_frame()
df.rename(columns={0: "tweet_id"}, inplace=True)
# df["id"] = df.index
df["tweet_id"] = df['tweet_id'].astype('Int64')
df["user_id"] = df.index
# list(df.columns)

df.reset_index(inplace=True, drop=True)
# df.rename(columns={"id": "user_id"}, inplace=True)
df.to_csv("../data/graph.edgelist", sep=" ", header=False, index=False)